In [1]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import threading
import cv2
import time
import numpy as np
from rclpy.qos import QoSProfile, QoSReliabilityPolicy
from message_filters import Subscriber, ApproximateTimeSynchronizer
from geometry_msgs.msg import PoseStamped
import json

In [2]:
# 只初始化一次
if not rclpy.ok():
    rclpy.init()

class RGBDSubscriber(Node):
    def __init__(self):
        super().__init__('rgbd_subscriber')
        self.bridge = CvBridge()
        self.latest_rgb = None
        self.latest_depth = None
        self.latest_end_pose = None
        self.latest_time = None

        # 创建三个 Subscriber（必须使用 message_filters.Subscriber）
        rgb_sub = Subscriber(self, Image, '/stereo/left/image_rect_color')
        depth_sub = Subscriber(self, Image, '/camera/depth/image_raw')
        end_pose_sub = Subscriber(self, PoseStamped, '/end_pose_stamped')

        # 三路 ApproximateTimeSynchronizer（时间容差 0.1 秒）
        self.ts = ApproximateTimeSynchronizer(
            [rgb_sub, depth_sub, end_pose_sub],
            queue_size=10,
            slop=0.1  # 允许最大 0.1 秒时间差
        )
        self.ts.registerCallback(self.sync_callback)

        self.get_logger().info("Subscribed to RGB, Depth, and /end_pose_stamped with ApproximateTimeSynchronizer")

    def sync_callback(self, rgb_msg, depth_msg, end_pose_msg):
        try:
            # 转 RGB 图像
            rgb_cv = self.bridge.imgmsg_to_cv2(rgb_msg, "bgr8")
            self.latest_rgb = rgb_cv

            # 转深度图像
            self.latest_depth = self.bridge.imgmsg_to_cv2(depth_msg, "passthrough")

            # 保存末端位姿
            self.latest_end_pose = end_pose_msg

            # 保存时间（可选）
            self.latest_time = rgb_msg.header.stamp.sec + rgb_msg.header.stamp.nanosec * 1e-9


            # 💡 在这里调用你的视觉检测 + 手眼变换逻辑
            # e.g., detect_object_and_transform_to_base(self.latest_rgb, self.latest_depth, pos, quat)

        except Exception as e:
            self.get_logger().error(f'Error in sync_callback: {e}')

# 创建节点
rgbd_sub = RGBDSubscriber()

# 启动非阻塞 spin 线程
def spin_thread():
    rclpy.spin(rgbd_sub)

spinner = threading.Thread(target=spin_thread, daemon=True)
spinner.start()
time.sleep(2)  # 等待几秒钟以确保接收到数据

[INFO] [1766719576.267223304] [rgbd_subscriber]: Subscribed to RGB, Depth, and /end_pose_stamped with ApproximateTimeSynchronizer


In [9]:
if rgbd_sub.latest_rgb is not None and rgbd_sub.latest_depth is not None:
    new_img = rgbd_sub.latest_rgb
    new_depth = rgbd_sub.latest_depth
    cv2.imwrite("img/new_img.png", new_img)
    depth_norm = cv2.normalize(new_depth, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    cv2.imwrite("img/new_depth_norm.png", depth_norm)
    cv2.imwrite("img/new_depth_raw.tiff", new_depth)  # TIFF 支持 float32
    # 用json保存pose
    pose = rgbd_sub.latest_end_pose.pose
    pos = [pose.position.x, pose.position.y, pose.position.z]
    quat = [pose.orientation.x, pose.orientation.y, pose.orientation.z, pose.orientation.w]
    with open("img/new_end_pose.json", "w") as f:
        json.dump({"position": pos, "quaternion": quat}, f, indent=4)
else:
    print("No synchronized data received yet.")